In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
import joblib
import re
from num2words import num2words


In [2]:
#loading the data
sheet=pd.ExcelFile(r'Product Matching Dataset.xlsx')
sheetName=sheet.sheet_names

master=sheet.parse(sheetName[0])
data=sheet.parse(sheetName[1])

In [3]:
#doing some analysis
data.nunique()

sku                              500
marketplace_product_name_ar      500
seller_item_name               49556
price                            198
dtype: int64

In [4]:
master.nunique()

sku                1000
product_name       1000
product_name_ar    1000
price               369
dtype: int64

In [6]:
column1_values = master['sku']

not_found = master[~master['sku'].isin(data['sku'])]

not_found.nunique()

sku                500
product_name       500
product_name_ar    500
price              286
dtype: int64

now i have main points:
1) data do not covel enough english names
2) data do not cover all sku from master file

In [ ]:
#cleaning and preparing the data

new = pd.DataFrame(columns=['sku', 'name', 'price', 'input'])

# Assign values from 'master' DataFrame and concatenate properly
new['sku'] = pd.concat([master['sku'], master['sku']], ignore_index=True)
new['name'] = pd.concat([master['product_name'], master['product_name_ar']], ignore_index=True)
new['price'] = pd.concat([master['price'], master['price']], ignore_index=True)

#add values to the things not in the dataset
expanded_rows = []

for _, row in new.iterrows():
    words = row['name'].split()  # Split product name into words
    for i in range(1, len(words) + 1):  # Generate incremental input values
        expanded_rows.append({
            'sku': row['sku'],
            'name': row['name'],
            'price': row['price'],
            'input': ' '.join(words[:i])  # Take first i words
        })

# Convert list to DataFrame
y = pd.DataFrame(expanded_rows)

# Rename columns from `data` to match `maybe_final`
data = data.rename(columns={
    "sku": "sku",
    "seller_item_name": "input",   # Seller name → Input
    "price": "price",
    "marketplace_product_name_ar": "name"  # Product name → Name
})

# Filter matching rows (sku & price must be the same)
merged_data = data[['sku', 'name', 'price', 'input']]

# Append to `maybe_final`
maybe_final = pd.concat([y, merged_data], ignore_index=True)

# Remove duplicates
clean_data = maybe_final.drop_duplicates()


In [9]:

# Save the updated file
clean_data.to_excel("clean_data.xlsx", index=False)

print("Data saved successfully!")

Data saved successfully!


In [2]:
#functions needed for preprocessing
def convert_numbers_to_text(text):
    # This function converts all numbers in the text to their word equivalents
    def replace_number(match):
        number = match.group()
        try:
            # Convert Arabic numerals to English numerals
            number = number.translate(str.maketrans('٠١٢٣٤٥٦٧٨٩', '0123456789'))
            # Convert the number to words
            return num2words(int(number), lang='en')
        except ValueError:
            return number  # Return the original if conversion fails

    # Use regex to find all numbers (including Arabic numerals) and replace them
    return re.sub(r'\d+', replace_number, text)


def preprocess_text(text):
    if pd.isna(text):
        return ""
    
    # Convert to lowercase
    text = text.lower()

    # Convert numbers to text
    text = convert_numbers_to_text(text)
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Strip leading/trailing whitespace
    text = text.strip()
    
    return text


In [11]:
my_data = pd.read_excel("clean_data.xlsx")

my_data['input'] = my_data['input'].apply(preprocess_text)
my_data['price'] = my_data['price'].astype(int)

my_data['combined_features'] = my_data['input'] + ' ' + my_data['price'].astype(str)


In [13]:
x = my_data['combined_features']
y = my_data['sku']

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the vectorizer
X = vectorizer.fit_transform(x)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# Initialize K-Nearest Neighbors model
model = KNeighborsClassifier(n_neighbors=3 , weights='distance', metric='cosine')


# Fit the model
model.fit(X_train, y_train)


# Predict the labels
y_pred = model.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Generate the classification report
class_report = classification_report(y_test, y_pred)

print("Classification Report:")
print(class_report)


Accuracy: 0.97
Classification Report:
              precision    recall  f1-score   support

           2       1.00      1.00      1.00         3
           4       0.97      1.00      0.98        29
           5       1.00      0.97      0.98        32
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         2
           9       1.00      0.96      0.98        25
          10       0.94      1.00      0.97        33
          11       1.00      1.00      1.00         3
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00        27
          15       0.88      0.96      0.92        24
          16       1.00      1.00      1.00         2
          18       1.00      1.00      1.00         2
          19       1.00      1.00      1.00         2
          20       1.00      0.93      0.96        28
          22       1.00      1.00      1.00        37
          25       1.00      0.50      0.67

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklea

In [14]:
#with this high accuracy, we need cross validation to make sure the model is not overfitting

scores = cross_val_score(model, X, y, cv=5)
print("Mean Accuracy:", scores.mean())

print("Cross-validation scores:", scores)
print("Standard deviation:", scores.std())


Mean Accuracy: 0.9434552696158963
Cross-validation scores: [0.93428499 0.94830644 0.94162865 0.95362392 0.93943234]
Standard deviation: 0.006790433733913319


as we get similar scores and small std, we can be sure that the model is not overfitting

In [15]:
#now we save the model and vectorizer to be used

joblib.dump(model, 'model.pkl')

joblib.dump(vectorizer, 'vectorizer.pkl')

print("Model and vectorizer saved successfully!")

Model and vectorizer saved successfully!


In [ ]:
## Load your trained model and vectorizer
model = joblib.load("model.pkl") 
vectorizer = joblib.load("vectorizer.pkl")  

def get_similarity_and_confidence(new_item_name, model, vectorizer):
    new_item_vector = vectorizer.transform([new_item_name])
    predicted_sku = model.predict(new_item_vector)
    
    #Get distances to nearest neighbors
    distances, _ = model.kneighbors(new_item_vector)
    similarity_score = 1 - distances.min()


    confidence_level = min(model.predict_proba(new_item_vector)[0].max() , similarity_score)

    
    if confidence_level > 0.95:
        note = "High"

    elif confidence_level > 0.75 and confidence_level <= 0.95:
        note = "Medium"
    else:
        note = "Low"
  
    
    return predicted_sku[0], similarity_score, confidence_level , note

def process_file(file_path):
    try:
        df = pd.read_excel(file_path)
        
        if 'input' not in df.columns:
            raise ValueError("The input column is missing in the Excel file.")
        
        df['price'] = df['price'].astype(int)
        df['input'] = df['input'].apply(preprocess_text)
        df['combined_features'] = df['input'] + ' ' + df['price'].astype(str)
        
        # Compute predictions and similarity metrics
        results = df['combined_features'].apply(lambda x: get_similarity_and_confidence(x, model, vectorizer))
        df[['Predicted SKU', 'Similarity Score', 'Confidence Level', ' Note']] = pd.DataFrame(results.tolist(), index=df.index)
        #for the debugging
        #df['true_positive'] = df['sku'] == df['Predicted SKU']

        output_file = "predictions_master.xlsx"
        df.to_excel(output_file, index=False)
        print(f"Predictions saved as {output_file}")
    except Exception as e:
        print(f"Error: {e}")

###########################
file_path = "new_master.xlsx"  # Replace with the path to your file
process_file(file_path)

Predictions saved as predictions_master.xlsx


- Master file accuracy: 99.8% (4 wrong of 2000 "marked between medium and low")
- Time needed for 2000 item: 31s
- confidence needed check in prediction (medium and low): 177 of 2000 about 8.8% of the dataset to be checked